In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from noteify.core.config import SAMPLE_RATE
from noteify.core.datasets import (MaestroDataset, MaestroDatasetProcessed, MusicAugmentor,
                                   MusicSegmentSampler, get_music_dataloader)
from noteify.core.training import (make_optimizer, make_scheduler, train_model,
                                   save_training_session, load_training_session,
                                   set_optimizer_lr)
from noteify.core.models import TranscriptionNN
from noteify.core.utils import plot_audio, plot_roll_info
from noteify.utils import get_rel_pkg_path

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("Warning: Could not find GPU! Using CPU only")

Using the GPU!


In [3]:
data_dir = get_rel_pkg_path("maestro/maestro-v3.0.0")
weights_dir = get_rel_pkg_path("weights/")
session_dir = get_rel_pkg_path("sessions/")

In [4]:
raw_dataset_train = MaestroDataset(data_dir, train=True)
raw_dataset_test = MaestroDataset(data_dir, train=False)

 10%|███████▉                                                                        | 96/962 [00:00<00:00, 959.20it/s]

Loading audio


100%|██████████████████████████████████████████████████████████████████████████████| 177/177 [00:00<00:00, 1053.74it/s]

Loading audio


In [5]:
batch_size = 32

In [6]:
dataset_train = MaestroDatasetProcessed(raw_dataset_train, augmentor=MusicAugmentor())
sampler_train = MusicSegmentSampler(dataset_train, batch_size, num_batches=None, random_start_times=True)
dataloader_train = get_music_dataloader(dataset_train, sampler_train, num_workers=8, pin_memory=True)

dataset_test = MaestroDatasetProcessed(raw_dataset_test)
sampler_test = MusicSegmentSampler(dataset_test, batch_size)
dataloader_test = get_music_dataloader(dataset_test, sampler_test, num_workers=None, pin_memory=True)

dataloaders = {'train': dataloader_train, 'test': dataloader_test}

In [7]:
model = TranscriptionNN()
model = model.to(device)
past_train_weights = os.path.abspath(os.path.join(weights_dir, "Experiment 12-08-2020 03-48PM/Weights Best.pckl"))
model.load_state_dict(torch.load(past_train_weights))

CQT kernels created, time used = 0.1221 seconds


<All keys matched successfully>

In [8]:
optimizer = make_optimizer(model, lr=0.0005)

In [9]:
# _ = load_training_session(model, optimizer,
#                           os.path.join(session_dir, "Session 12-08-2020 03-18PM"))

In [10]:
num_epochs = 100
scheduler = make_scheduler(optimizer, [60], gamma=0.5)

In [ ]:
tracker = train_model(device=device,
                      model=model,
                      dataloaders=dataloaders,
                      optimizer=optimizer,
                      lr_scheduler=scheduler,
                      save_log=True,
                      save_model=True,
                      save_dir=weights_dir,
                      save_best=True,
                      save_all=False,
                      num_epochs=num_epochs)

  0%|                                                                                         | 0/2964 [00:00<?, ?it/s]

Epoch 0/99
----------


Avg. Loss: 0.0849, Total Loss: 0.1052, Loss Parts: [0.0852, 0.0074, 0.0126]: 100%|█| 2964/2964 [51:05<00:00,  1.03s/it]
  0%|                                                                                          | 0/374 [00:00<?, ?it/s]

Training Loss: 0.0849


  2%|█▉                                                                                | 9/374 [00:47<31:49,  5.23s/it]


...

  2%|█▉                                                                                | 9/374 [00:44<29:48,  4.90s/it]


Test statistics: {'frame_avg_precision': 0.8219343761389408, 'reg_onset_mae': 0.1209413930773735, 'reg_offset_mae': 0.14012114703655243}


  0%|                                                                                         | 0/2964 [00:00<?, ?it/s]


Epoch 9/99
----------


Avg. Loss: 0.0765, Total Loss: 0.0922, Loss Parts: [0.0699, 0.0094, 0.0129]:  40%|▍| 1181/2964 [22:34<39:47,  1.34s/it]

In [ ]:
%qtconsole

In [ ]:
save_training_session(model, optimizer, session_dir)